In [194]:
import pandas as pd
import ast
import numpy as np

In [195]:
path = 'historical.csv'
df = pd.read_csv(path, index_col=0)[['premise', 'hypothesis', 'prompt', 'rationale']]

In [196]:
df['hypothesis'] = df['hypothesis'].apply(ast.literal_eval)

In [197]:
df.rationale = df.rationale.astype(str)
df['split'] = df.rationale.apply(lambda x: x.lower().split('most likely answer'))
df.split = df.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [198]:
df.head(1)

,premise,hypothesis,prompt,rationale,split
0,The morbidly obese man with unkempt facial hai...,"[french fries, enjoy, nausea, satisfaction, gas]",Questions: The morbidly obese man with unkempt...,1. The most likely answer is **(D) satisfactio...,is **(d) satisfaction**. the question is base...


In [199]:
# get correct index
df['correct_index'] = [{} for _ in range(len(df))]
for index, row in df.iterrows():
    for choice in row['hypothesis']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice.lower())

In [200]:
df['LLM_answer'] = df.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [201]:
df[df.LLM_answer.isna()]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer


In [202]:
# read paper's answer
paper = pd.read_csv('paper.csv', index_col=0)

In [203]:
paper['premise'] = paper.input.str.split('\n').apply(lambda x: x[0])
paper.set_index('premise', inplace=True)
paper.index = paper.index.str.strip()

In [204]:
df.set_index('premise', inplace=True)

In [205]:
paper['my_label'] = df['LLM_answer']

In [206]:
sum(paper.label == paper.my_label) / len(paper)

0.8074119700236115

In [207]:
df.to_csv('historical - full.csv')

In [238]:
TYPE = 'neutral'
temp = pd.read_csv(f'{TYPE} - full.csv')

import re
temp[~temp['rationale'].apply(lambda x: bool(re.match(r'\d+\.', x.strip())))][9:20]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer
3631,Where do you keep your beer at home?,"['sporting event', 'bowling alley', 'bucket', ...",Questions: Where do you keep your beer at home...,32°F. This prevents the food from spoiling for...,32°f. this prevents the food from spoiling for...,{'fridge': 470},fridge
3632,Where do you live if there are a lot of mailbo...,"['spacious house', 'mansion', 'building', 'apa...",Questions: Where do you live if there are a lo...,46°F. You can keep your beer in the fridge at ...,46°f. you can keep your beer in the fridge at ...,"{'building': 196, 'apartment': 112}",apartment
3633,Where do you look to see a meteor?,"['outerspace', 'orbit', 'universe', 'sky', 'ob...","Questions: Where do you look to see a meteor?,...",6°F and 46°F. You can keep your beer in the fr...,6°f and 46°f. you can keep your beer in the fr...,"{'sky': 1119, 'observatory': 1463}",sky
3634,Where do you need to use a fan?,"['hot room', 'sports stadium', 'appliance stor...","Questions: Where do you need to use a fan?, Ch...",8°C or 36°F and 46°F. You can keep your beer i...,8°c or 36°f and 46°f. you can keep your beer i...,{'hot room': 2070},hot room
3738,You go through a lot of training to be a soldi...,"['follow orders', 'wait for orders', 'follow i...",Questions: You go through a lot of training to...,"37 or 38 slots, each with a number and a color...","37 or 38 slots, each with a number and a color...","{'follow orders': 941, 'wait for orders': 956,...",fight for freedom
3763,A bugle sounded every morning where?,"['music store', 'army corps', 'army', 'militar...",Questions: A bugle sounded every morning where...,"100 centavos make a mexican peso, which is the...","100 centavos make a mexican peso, which is the...",{'military base': 3252},military base
3772,A city like Detroit ended up with a lot of aba...,"['forest', 'countryside', 'wilderness', 'subur...",Questions: A city like Detroit ended up with a...,10th floor is in what kind of structure? A sky...,10th floor is in what kind of structure? a sky...,{'suburbs': 2362},suburbs
3844,"The toddler is awaking from a nap, what doesn'...","['nervous', 'eyes opening', 'getting out of be...","Questions: The toddler is awaking from a nap, ...",2AM and the line hadn't moved is because there...,2am and the line hadn't moved is because there...,{'tiredness': 184},tiredness
4498,What is a farmer growing in Illinois or Iowa?,"['grow plants', 'seed plants', 'seed crops', '...",Questions: What is a farmer growing in Illinoi...,6 million. Beaver are the national animal of C...,6 million. beaver are the national animal of c...,{'raise corn': 1359},raise corn
4617,Sam treasured his freedom. When he became a P...,"['communism', 'oppression', 'slavery', 'bad ma...",Questions: Sam treasured his freedom. When he...,100 years**. The question asks how long Sam ha...,100 years**. the question asks how long sam ha...,{'captivity': 2127},captivity
